In [1]:
%%configure -f 
{"executorMemory": "12G", "executorCores": 6}

In [2]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy'])

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
distance_calculation_df = (
    spark
    .read
    .format("mongodb")
    .option("database", "enhancer3d")
    .option("collection", "distance_calculation")
    .load()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql import functions as F, types as T
import numpy as np

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
cell_line_1 = 'GM12878'
cell_line_2 = 'HFFC6'
species = 'Nean'

gm12878_neanderthal_df = (
    distance_calculation_df
    .select('_id.*', '*')
    .drop('_id')
    .where('ensemble_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
DataFrame.filter() missing 1 required positional argument: 'condition'
Traceback (most recent call last):
TypeError: DataFrame.filter() missing 1 required positional argument: 'condition'



In [15]:
%%pretty
gm12878_neanderthal_df.show(n=1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

project_id,ensemble_id,region_id,gene_id,enh_id,avg_dist,dist,enh_center_pos,enh_center_position,enh_chr,enh_end,enh_loci,enh_model_coloring_end,enh_model_coloring_start,enh_model_position,enh_score,enh_start,enh_tSS_distance,gene_TSS_pos,gene_chr,gene_end,gene_model_coloring_end,gene_model_coloring_start,gene_model_position,gene_start,gene_strand,number_bins,project_authors,project_cell_lines,project_executed_at,project_species,region_chr,region_end,region_start,var_dist
8k_models_project...,models3D_GM12878_...,chr10:10300866-12...,ENSG00000065665.21,chr10:10854747-10...,32.353797912597656,[41.3013191223144...,10855122,10855122,chr10,10855497,chr10:10854747-10...,178,178,178,8.12594160720854,10854747,1276440,12131562,chr10,12171887,1495,1454,1454,12131562,+,100,"[Nikita Kozlov, M...",[GM12878],2025-04-06 18:27:...,"[Denisovans, Nean...",chr10,12302793,10300866,77.25293731689453
